In [ ]:
from ib_insync import*
util.startLoop()

In [ ]:
class VolumeRsiStrategy:
    def __init__(self,ticker):
        self.ticker = ticker
        self.ib = IB()
        self.volume_treshold = 1000000
        self.rsi_treshold = 70
        self.momentum_treshold = 0.5
        self.position = None
        self.trading_signal = self.generate_trading_signal()
        
    def generate_trading_signal(self):
        contract =Stock(self.ticker,'SMART','USD')
        self.ib.reqMktData(contract,'',snapshot=False)
        self.ib.sleep(2)
        
        while True:
            ticker = self.ib.ticker(contract)
            rsi = self.ib.rsi(ticker.close, timeperiod = 14)
            momentum = self.calculate_momentum(ticker.close,timeperiod = 10)
            
            if (
                ticker.volume[-1]>self.volume_treshold and
                rsi[-1]>self.rsi_treshold and
                momentum > self.momentum_treshold
            ):
                yield 'BUY'
                
            else:
                yield 'SELL'
                
    def calculate_momentum(self,prices,timeperiod):
        returns = [((prices[i] - prices[i-timeperiod])/prices[i-timeperiod]) for i in range(timeperiod,len(prices))]
        momentum = sum(returns)/len(returns)
        return momentum
        
    def calculate_order_quantity(self,total_capital):
        return int(total_capital*0.5)
    
    def execute_trade(self,trading_signal):
        if trading_signal == 'BUY':
            contract = Stock(self.ticker,'SMART','USD')
            total_capital = self.ib.accountSummary()[0].value
            order_quantity = self.calculate_order_quantity(total_capital)
            order = MarketOrder('BUY',order_quantity)
            trade = self.ib.placeOrder(contract,order)
            self.position = trade.contract
            print('Placed order:',trade)
            
        elif trading_signal == 'SELL':
            if self.position is not None:
                self.ib.placeOrder(self.position.contract,MarketOrder('SELL',self.position.position))
                self.position = None
                print('Sold position')
                
    def run_strategy(self):
        self.ib.connect('XXXXXXX',XXXX,clientId=XXXX)
        
        trading_signal = next(self.trading_signal)
        print('Initial trading signal',trading_signal)
        
        self.ib.run()
        
        self.ib.disconnect()
        
    def update_market_data(self, tickers):
        for ticker in tickers:
            if ticker.contract == self.position:
                self.check_profit_loss(ticker.close)
        self.ib.sleep(1)
        
        
strategy = VolumeRsiStrategy('AAPL')
strategy.ib.pendingTickersEvent += strategy.update_market_data
strategy.execute_trade(next(strategy.trading_signal))
strategy.run_strategy() 
strategy.disconnect()